In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, precision_score
#from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier

from imblearn.over_sampling import SMOTE

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [2]:
# reading in full data set
brfss_total = pd.read_csv("../csv_data/brfss_total.csv")

In [3]:
brfss_total.head()

,Unnamed: 0,_STATE,DISPCODE,PHYSHLTH,MENTHLTH,USENOW3,HISPANC2,MARITAL,CHILDREN,EMPLOY,...,ACETTHEM,ACEHVSEX,MSCODE,_IMPAGE,_RFHLTH,_SMOKER3,_PRACE,_EDUCAG,_INCOMG,_TOTINDA
0,0,5.0,110.0,0.0,0.0,3.0,2.0,1.0,1.0,2.0,...,1.0,1.0,5.0,53.0,1.0,4.0,1.0,3.0,5.0,1.0
1,1,5.0,110.0,15.0,0.0,3.0,2.0,2.0,0.0,0.0,...,1.0,1.0,5.0,64.0,2.0,3.0,1.0,2.0,2.0,1.0
2,2,5.0,110.0,6.0,0.0,3.0,2.0,1.0,0.0,0.0,...,1.0,1.0,5.0,58.0,1.0,4.0,1.0,3.0,2.0,1.0
3,3,5.0,110.0,30.0,0.0,3.0,2.0,1.0,0.0,0.0,...,1.0,1.0,5.0,76.0,2.0,4.0,1.0,1.0,0.0,2.0
4,4,5.0,110.0,13.0,0.0,3.0,2.0,3.0,0.0,0.0,...,1.0,0.0,5.0,82.0,2.0,3.0,1.0,2.0,0.0,2.0


In [4]:
# just making sure no nulls are present
brfss_total.isna().sum()

Unnamed: 0    0
_STATE        0
DISPCODE      0
PHYSHLTH      0
MENTHLTH      0
USENOW3       0
HISPANC2      0
MARITAL       0
CHILDREN      0
EMPLOY        0
RENTHOM1      0
SEX           0
QLACTLM2      0
ACEDEPRS      0
ACEDRINK      0
ACEDRUGS      0
ACEPRISN      0
ACEDIVRC      0
ACEPUNCH      0
ACEHURT       0
ACESWEAR      0
ACETOUCH      0
ACETTHEM      0
ACEHVSEX      0
MSCODE        0
_IMPAGE       0
_RFHLTH       0
_SMOKER3      0
_PRACE        0
_EDUCAG       0
_INCOMG       0
_TOTINDA      0
dtype: int64

In [5]:
brfss_total.drop(columns=['Unnamed: 0'], inplace=True)

**Variables I will try to predict with my models:**
- USENOW3: Do you currently use chewing tobacco, snuff, or snus every day, some days, or not at all?
    - classification
    - 0 = Don't know, Not sure or Refused, 1 = every day, 2 = some days, 3 = not at all
- QLACTLM2: Are you limited in any way in any activities because of physical, mental, or emotional problems?
    - classification
    - 0 = Don't know, Not sure or Refused, 1 = yes, 2 = no
- _RFHLTH: Adults with good or better health vs. fair or poor health
    - classification
    - based off of GENHLTH
    - 0 = Don't know, Not sure or Refused, 1 = Good or Better Health, 2 = Fair or Poor Health
- _SMOKER3: Four-level smoker status: Everyday smoker, Someday smoker, Former smoker, Non-smoker
    - classification
    - based off of SMOKE100 & SMOKEDAY
    - 0 = Don't know, Not sure or Refused, 1 = Current smoker (now smokes every day), 2 = Current smoker (now smokes some days), 3 = Former smoker, 4 = Never smoked
 
 
**Will OneHotEncode/ dummify ordinal/nominal features**

**Will use SMOTE to compensensate for imbalanced classes**

**Will aggregate all ACEs into two groups: Abuse and Household Challenges**

In [6]:
np.random.seed(151)

In [7]:
# creating X variable with all features
X_all = brfss_total.drop(columns=['USENOW3', 'QLACTLM2', '_RFHLTH', '_SMOKER3'])

In [8]:
# creating the 4 y's
y_tobacco = brfss_total['USENOW3']
y_activity = brfss_total['QLACTLM2']
y_health = brfss_total['_RFHLTH']
y_smoker = brfss_total['_SMOKER3']

In [9]:
#original baseline for tobacco
y_tobacco.value_counts(normalize=True)

3.0    0.966560
1.0    0.017872
2.0    0.012990
0.0    0.002578
Name: USENOW3, dtype: float64

In [10]:
#original baseline for activity
y_activity.value_counts(normalize=True)

2.0    0.724520
1.0    0.268997
0.0    0.006482
Name: QLACTLM2, dtype: float64

In [11]:
#original baseline for health
y_health.value_counts(normalize=True)

1.0    0.814742
2.0    0.181991
0.0    0.003267
Name: _RFHLTH, dtype: float64

In [12]:
#original baseline for smoker
y_smoker.value_counts(normalize=True)

4.0    0.521569
3.0    0.308324
1.0    0.119604
2.0    0.044226
0.0    0.006278
Name: _SMOKER3, dtype: float64

In [13]:
# splitting X up so I can do some engineering on the nominal data and ACE columns
X_num = X_all[['PHYSHLTH', 'MENTHLTH', 'CHILDREN']]
X_cat = X_all[['_STATE', 'DISPCODE', 'HISPANC2', 'MARITAL', 'EMPLOY', 'RENTHOM1', 'SEX', 'MSCODE', 
               '_IMPAGE', '_PRACE', '_EDUCAG', '_INCOMG','_TOTINDA']]
ace = X_all[['ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN', 'ACEDIVRC', 'ACEPUNCH', 'ACEHURT', 'ACESWEAR', 
                       'ACETOUCH', 'ACETTHEM', 'ACEHVSEX']]

In [14]:
# updating ACE columns to be a count depending on the question
# first 6 questions are yes or no, so yes will be be counted as 1 and no will be counted as 0
# last 5 are questions of frequency, never = 0, once = 1, more than once will equal 2 (since not given an exact number)
ace['ACEDEPRS'] = ace['ACEDEPRS'].map({1:1, 2:0, 0:0})
ace['ACEDRINK'] = ace['ACEDRINK'].map({1:1, 2:0, 0:0})
ace['ACEDRUGS'] = ace['ACEDRUGS'].map({1:1, 2:0, 0:0})
ace['ACEPRISN'] = ace['ACEPRISN'].map({1:1, 2:0, 0:0})
ace['ACEDIVRC'] = ace['ACEDIVRC'].map({1:1, 2:0, 0:0})
ace['ACEPUNCH'] = ace['ACEPUNCH'].map({1:0, 2:1, 3:2})
ace['ACEHURT'] = ace['ACEHURT'].map({1:0, 2:1, 3:2, 0:0})
ace['ACESWEAR'] = ace['ACESWEAR'].map({1:0, 2:1, 3:2, 0:0})
ace['ACETOUCH'] = ace['ACETOUCH'].map({1:0, 2:1, 3:2, 0:0})
ace['ACETTHEM'] = ace['ACETTHEM'].map({1:0, 2:1, 3:2, 0:0})
ace['ACEHVSEX'] = ace['ACEHVSEX'].map({1:0, 2:1, 3:2, 0:0})

In [15]:
X_num['ACE_Count'] = ace.sum(axis = 1)

In [16]:
X_cat = X_cat.astype(str)

In [17]:
# dummifying nominal variables for X_all
X_dummies = pd.get_dummies(X_cat, drop_first=True)
X_dummies.head()

,_STATE_15.0,_STATE_19.0,_STATE_22.0,_STATE_27.0,_STATE_30.0,_STATE_32.0,_STATE_37.0,_STATE_40.0,_STATE_47.0,_STATE_5.0,...,_EDUCAG_2.0,_EDUCAG_3.0,_EDUCAG_4.0,_INCOMG_1.0,_INCOMG_2.0,_INCOMG_3.0,_INCOMG_4.0,_INCOMG_5.0,_TOTINDA_1.0,_TOTINDA_2.0
0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1


In [18]:
X_num.head()

,PHYSHLTH,MENTHLTH,CHILDREN,ACE_Count
0,0.0,0.0,1.0,0.0
1,15.0,0.0,0.0,3.0
2,6.0,0.0,0.0,0.0
3,30.0,0.0,0.0,0.0
4,13.0,0.0,0.0,0.0


In [19]:
# merging numerical and nominal data into one data frame
X_all = X_num.merge(X_dummies, left_index=True, right_index=True)

In [20]:
X_all.shape

(117555, 138)

In [21]:
# to compensate for unbalanced classes in my y's will use SMOTE

sm = SMOTE(random_state=151)
X_all1, y_tobacco = sm.fit_resample(X_all, y_tobacco)

sm2 = SMOTE(random_state=151)
X_all2, y_activity = sm2.fit_resample(X_all, y_activity)

sm3 = SMOTE(random_state=151)
X_all3, y_health = sm3.fit_resample(X_all, y_health)

sm4 = SMOTE(random_state=151)
X_all4, y_smoker = sm4.fit_resample(X_all, y_smoker)

In [22]:
# new baseline for tobacco
y_tobacco.value_counts(normalize=True)

3.0    0.25
0.0    0.25
2.0    0.25
1.0    0.25
Name: USENOW3, dtype: float64

In [23]:
# looks like SMOTE has increased the size of my y's more than 4x, so will probably take some time for models to run
y_tobacco.shape

(454496,)

In [24]:
# new baseline for activity
y_activity.value_counts(normalize=True)

1.0    0.333333
0.0    0.333333
2.0    0.333333
Name: QLACTLM2, dtype: float64

In [25]:
# new baseline for health
y_health.value_counts(normalize=True)

0.0    0.333333
2.0    0.333333
1.0    0.333333
Name: _RFHLTH, dtype: float64

In [26]:
# new baseline for smoker
y_smoker.value_counts(normalize=True)

1.0    0.2
3.0    0.2
0.0    0.2
2.0    0.2
4.0    0.2
Name: _SMOKER3, dtype: float64

In [27]:
X_all1.shape

(454496, 138)

In [28]:
# creating training and testing sets for all y's (stratified on y, but since the classes are equal probably didn't have to)
X_train_all, X_test_all, y_train_tobacco, y_test_tobacco = train_test_split(X_all1, y_tobacco, random_state = 151, stratify=y_tobacco)
X_train_all2, X_test_all2, y_train_activity, y_test_activity = train_test_split(X_all2, y_activity, random_state = 151, stratify=y_activity)
X_train_all3, X_test_all3, y_train_health, y_test_health = train_test_split(X_all3, y_health, random_state = 151, stratify=y_health)
X_train_all4, X_test_all4, y_train_smoker, y_test_smoker = train_test_split(X_all4, y_smoker, random_state = 151, stratify=y_smoker)

In [29]:
pipe_all_tobacco = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=300))
pipe_all_tobacco.fit(X_train_all, y_train_tobacco)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(min_samples_split=5,
                                        n_estimators=300))])

In [30]:
pipe_all_activity = make_pipeline(SelectKBest(f_classif, k=136), StandardScaler(), ExtraTreesClassifier(max_depth=None, n_estimators=500))
pipe_all_activity.fit(X_train_all2, y_train_activity)

Pipeline(steps=[('selectkbest', SelectKBest(k=136)),
                ('standardscaler', StandardScaler()),
                ('extratreesclassifier',
                 ExtraTreesClassifier(n_estimators=500))])

In [31]:
pipe_all_health = make_pipeline(SelectKBest(f_classif, k=106), StandardScaler(), RandomForestClassifier(max_depth=None, n_estimators=100))
pipe_all_health.fit(X_train_all3, y_train_health)

Pipeline(steps=[('selectkbest', SelectKBest(k=106)),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [32]:
pipe_all_smoker = make_pipeline(SelectKBest(f_classif, k=106), StandardScaler(), RandomForestClassifier(max_depth=None, n_estimators=500))
pipe_all_smoker.fit(X_train_all4, y_train_smoker)

Pipeline(steps=[('selectkbest', SelectKBest(k=106)),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=500))])

In [33]:
tobacco_all_preds = pipe_all_tobacco.predict(X_test_all)
activity_all_preds = pipe_all_activity.predict(X_test_all2)
health_all_preds = pipe_all_health.predict(X_test_all3)
smoker_all_preds = pipe_all_smoker.predict(X_test_all4)

tobacco_all_prec = precision_score(y_test_tobacco, tobacco_all_preds, average='micro')
activity_all_prec = precision_score(y_test_activity, activity_all_preds, average='micro')
health_all_prec = precision_score(y_test_health, health_all_preds, average='micro')
smoker_all_prec = precision_score(y_test_smoker, smoker_all_preds, average='micro')

In [34]:
print(f' training accuracy for tobacco w/ACE grouped: {pipe_all_tobacco.score(X_train_all, y_train_tobacco)}')
print(f' training accuracy for activity w/ACE grouped: {pipe_all_activity.score(X_train_all2, y_train_activity)}')
print(f' training accuracy for health w/ACE grouped: {pipe_all_health.score(X_train_all3, y_train_health)}')
print(f' training accuracy for smoker w/ACE grouped: {pipe_all_smoker.score(X_train_all4, y_train_smoker)}')

MemoryError: Unable to allocate 359. MiB for an array with shape (138, 340872) and data type float64

In [35]:
print(f' testing accuracy for tobacco w/ACE grouped: {pipe_all_tobacco.score(X_test_all, y_test_tobacco)}')
print(f' testing accuracy for activity w/ACE grouped: {pipe_all_activity.score(X_test_all2, y_test_activity)}')
print(f' testing accuracy for health w/ACE grouped: {pipe_all_health.score(X_test_all3, y_test_health)}')
print(f' testing accuracy for smoker w/ACE grouped: {pipe_all_smoker.score(X_test_all4, y_test_smoker)}')

 testing accuracy for tobacco w/ACE grouped: 0.986560937830036


MemoryError: Unable to allocate 1.46 MiB for an array with shape (63879, 1, 3) and data type float64

In [ ]:
print(f'Precision for tobacco w/ACE grouped: {tobacco_all_rfc_prec}')
print(f'Precision for activity w/ACE grouped: {activity_all_rfc_prec}')
print(f'Precision for health w/ACE grouped: {health_all_rfc_prec}')
print(f'Precision for smoker w/ACE grouped: {smoker_all_rfc_prec}')

In [ ]:
X_ace = X_num['ACE_Count']

In [ ]:
# to compensate for unbalanced classes in my y's will use SMOTE

sm = SMOTE(random_state=151)
X1, y_tobacco = sm.fit_resample(X_ace, y_tobacco)

sm2 = SMOTE(random_state=151)
X2, y_activity = sm2.fit_resample(X_ace, y_activity)

sm3 = SMOTE(random_state=151)
X3, y_health = sm3.fit_resample(X_ace, y_health)

sm4 = SMOTE(random_state=151)
X4, y_smoker = sm4.fit_resample(X_ace, y_smoker)

In [ ]:
# creating training and testing sets for all y's (stratified on y, but since the classes are equal probably didn't have to)
X_train_ace, X_test_ace, y_train_tobacco, y_test_tobacco = train_test_split(X1, y_tobacco, random_state = 151, stratify=y_tobacco)
X_train_ace2, X_test_ace2, y_train_activity, y_test_activity = train_test_split(X2, y_activity, random_state = 151, stratify=y_activity)
X_train_ace3, X_test_ace3, y_train_health, y_test_health = train_test_split(X3, y_health, random_state = 151, stratify=y_health)
X_train_ace4, X_test_ace4, y_train_smoker, y_test_smoker = train_test_split(X4, y_smoker, random_state = 151, stratify=y_smoker)

In [ ]:
ace_tobacco = xgb.XGBClassifier(learning_rate = 0.5, max_depth = 3, n_estimators = 20)
ace_tobacco.fit(X_train_ace, y_train_tobacco)

In [ ]:
ace_activity = RandomForestClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 3, n_estimators = 100)
ace_activity.fit(X_train_ace2, y_train_activity)

In [ ]:
ace_health = ExtraTreesClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 500)
ace_health.fit(X_train_ace3, y_train_health)

In [ ]:
ace_smoker = ExtraTreesClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 300)
ace_smoker.fit(X_train_ace4, y_train_smoker)

In [ ]:
tobacco_ace_preds = ace_tobacco.predict(X_test_ace)
activity_ace_preds = ace_activity.predict(X_test_ace2)
health_ace_preds = ace_health.predict(X_test_ace3)
smoker_ace_preds = ace_smoker.predict(X_test_ace4)

tobacco_ace_prec = precision_score(y_test_tobacco, tobacco_ace_preds, average='micro')
activity_ace_prec = precision_score(y_test_activity, activity_ace_preds, average='micro')
health_ace_prec = precision_score(y_test_health, health_ace_preds, average='micro')
smoker_ace_prec = precision_score(y_test_smoker, smoker_ace_preds, average='micro')

In [ ]:
print(f' training accuracy for tobacco: {ace_tobacco.score(X_train_ace, y_train_tobacco)}')
print(f' training accuracy for activity: {ace_activity.score(X_train_ace2, y_train_activity)}')
print(f' training accuracy for health: {ace_health.score(X_train_ace3, y_train_health)}')
print(f' training accuracy for smoker: {ace_smoker.score(X_train_ace4, y_train_smoker)}')

In [ ]:
print(f' testing accuracy for tobacco: {ace_tobacco.score(X_test_ace, y_test_tobacco)}')
print(f' testing accuracy for activity: {ace_activity.score(X_test_ace2, y_test_activity)}')
print(f' testing accuracy for health: {ace_health.score(X_test_ace3, y_test_health)}')
print(f' testing accuracy for smoker: {ace_smoker.score(X_test_ace4, y_test_smoker)}')

In [ ]:
print(f'Precision for tobacco: {tobacco_ace_prec}')
print(f'Precision for activity: {activity_ace_prec}')
print(f'Precision for health: {health_ace_prec}')
print(f'Precision for smoker: {smoker_ace_prec}')